In [ ]:
#Criando conjunto de "Dados Estruturados"

import pandas as pd
import numpy as np

# Importando Dados para o COLAB para fazer a montagem do drive no notebook
from google.colab import drive
drive.mount('/content/drive')

print('Importação completa!')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Notícias Verdadeiras
X = []
y = []

N = 3600 # Número de notícias coletadas do banco de "notícias genuínas"

i = 0
while i < N:
    try:
        file = open("/content/drive/MyDrive/Detector de FakeNews/dataset/true/"+str(i+1)+".txt")
        # Os arquivos contidos na pasta apresentam como identidade um número mais a extensão .txt
        X.append(file.readlines())
        y.append(True)
        file.close()
    except:
        print('Erro na leitura ou ausência do arquivo: ', i)
        # Sequencialmente, cada arquivos será checado, um a um
        X.append([None])
        y.append([None])
    i += 1

news = pd.DataFrame(data = list(zip(X, y)), columns = ['Conteúdo', 'Veredicto'])
news.index = np.arange(len(news))     
print('Validação completa')

Erro na leitura ou ausência do arquivo:  585
Erro na leitura ou ausência do arquivo:  1606
Validação completa


In [ ]:
#Notícias Falsas
X = []
y = []

N = 3600  # Número de notícias coletadas do banco de "notícias falsas"
i = 0
while i < N:
    try:
        file = open("/content/drive/MyDrive/Detector de FakeNews/dataset/fake/"+str(i+1)+".txt")
        # Os arquivos contidos na pasta apresentam como identidade um número mais a extensão .txt
        X.append(file.readlines())
        y.append(False)
        file.close()
    except:
        print('Erro na leitura ou ausência do arquivo: ', i)
        # Sequencialmente, cada arquivos será checado, um a um
        X.append([None])
        y.append([None])
    i += 1

fakenews = pd.DataFrame(data = list(zip(X, y)), columns = ['Conteúdo', 'Veredicto'])
fakenews.index = np.arange(len(fakenews)) + len(news)
print('Validação completa')


In [ ]:
# Dataset de notícias falsas

fakenews.head(10)

In [ ]:
#Tratando os Dados
#Subsituição de caracteres invalidos das noticias falsas
news_np = pd.DataFrame.to_numpy(fakenews['Conteúdo'])
lista = []
for i in range(len(news_np)):
  lista.append(news_np[i])
  if lista[i][0] != None:
    lista[i][0] = lista[i][0].replace('\n', '') #Removendo/alterando caracteres
    lista[i][0] = lista[i][0].replace('é', 'e')
    lista[i][0] = lista[i][0].replace('ê', 'e')
    lista[i][0] = lista[i][0].replace('á', 'a')
    lista[i][0] = lista[i][0].replace('â', 'a')
    lista[i][0] = lista[i][0].replace('à', 'a')
    lista[i][0] = lista[i][0].replace('ã', 'a')
    lista[i][0] = lista[i][0].replace('õ', 'o')
    lista[i][0] = lista[i][0].replace('ó', 'o')
    lista[i][0] = lista[i][0].replace('ô', 'o')
    lista[i][0] = lista[i][0].replace('ú', 'u')
    lista[i][0] = lista[i][0].replace('í', 'i')
    lista[i][0] = lista[i][0].replace('ç', 'c') 
  lista[i] = lista[i][0]
fakenews['Conteúdo'] = lista


In [ ]:
# Dataset de notícias verdadeiras

news.head(10)

NameError: ignored

In [ ]:
#Tratando os Dados
#Subsituição de caracteres invalidos das noticias verdadeiras 
news_np = pd.DataFrame.to_numpy(news['Conteúdo'])
lista = []
for i in range(len(news_np)):
  lista.append(news_np[i])
  if lista[i][0] != None:
    lista[i][0] = lista[i][0].replace('\n', '') #Removendo/alterando caracteres
    lista[i][0] = lista[i][0].replace('é', 'e')
    lista[i][0] = lista[i][0].replace('ê', 'e')
    lista[i][0] = lista[i][0].replace('á', 'a')
    lista[i][0] = lista[i][0].replace('â', 'a')
    lista[i][0] = lista[i][0].replace('à', 'a')
    lista[i][0] = lista[i][0].replace('ã', 'a')
    lista[i][0] = lista[i][0].replace('õ', 'o')
    lista[i][0] = lista[i][0].replace('ó', 'o')
    lista[i][0] = lista[i][0].replace('ô', 'o')
    lista[i][0] = lista[i][0].replace('ú', 'u')
    lista[i][0] = lista[i][0].replace('í', 'i')
    lista[i][0] = lista[i][0].replace('ç', 'c') 
  lista[i] = lista[i][0]
news['Conteúdo'] = lista


NameError: ignored

In [ ]:
news.describe()

,Conteúdo,Veredicto
count,3598,3600
unique,3597,2
top,Suplicy participara de programa de Doria na we...,True
freq,2,3598


In [ ]:
news.count()

Conteúdo     3598
Veredicto    3600
dtype: int64

In [ ]:
news = news.dropna() # Removendo os poucos valores inválidos
fakenews = fakenews.dropna()

In [ ]:
df = pd.concat([news, fakenews]) # Unindo conjunto fakenews e news
df.index = np.arange(len(df))

In [ ]:
df.head()

,Conteúdo,Veredicto
0,O Podemos decidiu expulsar o deputado federal...,True
1,Em evento realizado nesta terca-feira para div...,True
2,﻿Apos o prefeito de Manaus Arthur Virgilio (PS...,True
3,﻿Doria vai receber Ze Celso apos reuniao com r...,True
4,"Gustavo Pedreira Ferraz, que admitiu buscar ma...",True


In [ ]:
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score

In [ ]:
# Importando palavras mais comuns do Português

file = open('/content/drive/MyDrive/Detector de FakeNews/stopwords.txt')
stopwords = file.readlines()
for i in range(len(stopwords)):
  stopwords[i] = stopwords[i].replace(' ', '')
  stopwords[i] = stopwords[i].replace('\n', '')
  stopwords[i] = stopwords[i].replace('é', 'e')
  stopwords[i] = stopwords[i].replace('ê', 'e')
  stopwords[i] = stopwords[i].replace('á', 'a')
  stopwords[i] = stopwords[i].replace('â', 'a')
  stopwords[i] = stopwords[i].replace('à', 'a')
  stopwords[i] = stopwords[i].replace('ã', 'a')
  stopwords[i] = stopwords[i].replace('õ', 'o')
  stopwords[i] = stopwords[i].replace('ô', 'o')
  stopwords[i] = stopwords[i].replace('ó', 'o')
  stopwords[i] = stopwords[i].replace('ú', 'u')
  stopwords[i] = stopwords[i].replace('í', 'i')
  stopwords[i] = stopwords[i].replace('ç', 'c') 
file.close()


In [ ]:
stopwords = stopwords[:50]

In [ ]:
stopwords

In [ ]:
df['Conteúdo']

In [ ]:
corpus  = []
pstem = PorterStemmer()
for i in range(5):#len(df['Conteúdo'])):
    # Removendo alguns caracteres não desejados
    X = re.sub("[^a-zA-Z]", ' ', df['Conteúdo'][i])
    # Mudar para lowercase
    X = X.lower()
    X = X.split()
    #remover palavras listada entre as mais frequentes
    X = [pstem.stem(word) for word in X if not word in set(stopwords)]
    X = ' '.join(X)
    #Append cleaned tweet to corpus
    corpus.append(X)
    

In [ ]:
#model.predict(['Informação não validada'])
# True ou False
# Probabilidade [0, 1]


In [ ]:
df['Conteúdo'][0]

'O Podemos decidiu  expulsar o deputado federal Carlos Gaguim do partido apos a Policia Federal fazer buscas a apreensoes no gabinete dele na Camara. Com isso, a legenda abre espaco para receber a senadora expulsa pelo PMDB, Katia Abreu. Por meio de nota, a legenda informou que o afastamento do parlamentar ja era algo acordado entre os filiados da sigla.  “Ainda que o parlamentar tenha comunicado a conclusao de sua desfiliacao para esta semana, diante dos fatos noticiados hoje, a Executiva Nacional do Podemos solicita o imediato cancelamento de sua filiacao dos quadros do partido”.'

In [ ]:
df.to_excel('/content/drive/MyDrive/news.xlsx')